# Create Potential Field

In [ ]:
import os

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

In [ ]:
import json

In [ ]:
with open('config.json') as config:
    info = json.load(config)

nx = info['nx']
ny = info['ny']
nz = info['nz']
b_norm = info['b_norm']
spatial_norm = info['spatial_norm']

input_path = info['input_path']

In [ ]:
import pickle

In [ ]:
input_original = os.path.join(input_path, 'original')
b_bottom_original_path = os.path.join(input_original, "b_bottom_original.pickle")

with open(b_bottom_original_path, "rb") as f:
    b_bottom = pickle.load(f)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm

In [ ]:
from zpinn.pinn_nf2_cleanup import PotentialModel, create_coordinates

In [ ]:
bottom_values = b_bottom.reshape(-1, 3)
bottom_bounds = (0, nx-1, 0, ny-1, 0, 0)
bottom_coords = create_coordinates(bottom_bounds).reshape(-1, 3)
b_n = torch.tensor(bottom_values[:, 2], dtype=torch.float64)
r_p = torch.tensor(bottom_coords, dtype=torch.float64)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = nn.DataParallel(PotentialModel(b_n, r_p)).to(device)

In [ ]:
top_bounds = (0, nx-1, 0, ny-1, nz-1, nz-1)
lateral_bounds_1 = (0, 0, 0, ny-1, 0, nz-1)
lateral_bounds_2 = (nx-1, nx-1, 0, ny-1, 0, nz-1)
lateral_bounds_3 = (0, nx-1, 0, 0, 0, nz-1)
lateral_bounds_4 = (0, nx-1, ny-1, ny-1, 0, nz-1)

top_coords = create_coordinates(top_bounds).reshape(-1, 3)
lateral_coords_1 = create_coordinates(lateral_bounds_1).reshape(-1, 3)
lateral_coords_2 = create_coordinates(lateral_bounds_2).reshape(-1, 3)
lateral_coords_3 = create_coordinates(lateral_bounds_3).reshape(-1, 3)
lateral_coords_4 = create_coordinates(lateral_bounds_4).reshape(-1, 3)

In [ ]:
top_lateral_coordinates = [top_coords,
                           lateral_coords_1,
                           lateral_coords_2,
                           lateral_coords_3,
                           lateral_coords_4]

In [ ]:
pf_fields = []
pf_coords = []
for r_coords in top_lateral_coordinates:
    r_coords = torch.tensor(r_coords, dtype=torch.float64)
    # pf_batch_size = int(np.prod(r_coords.shape[:-1]) // 500)
    pf_batch_size = int(1024 * 512 ** 2 / (nx*ny))

    fields = []
    for r, in tqdm(DataLoader(TensorDataset(r_coords), batch_size=pf_batch_size, num_workers=2),
                        desc='Potential Boundary'):
        r = r.to(device).requires_grad_(True)
        p_batch = model(r)
        b_p = -1 * torch.autograd.grad(p_batch, r, torch.ones_like(p_batch), retain_graph=True, create_graph=True)[0]
        fields += [b_p.clone().detach().cpu().numpy()]
    pf_fields += [np.concatenate(fields)]
    pf_coords += [r_coords.clone().detach().cpu().numpy()]

Potential Boundary: 100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


In [ ]:
top_lateral_values = np.concatenate(pf_fields) 
top_lateral_coords = np.concatenate(pf_coords)

boundary_values = np.concatenate([top_lateral_values, bottom_values])
boundary_coords = np.concatenate([top_lateral_coords, bottom_coords])

normalized_boundary_values = boundary_values / b_norm
normalized_boundary_coords = boundary_coords / spatial_norm

boundary_data = np.stack([normalized_boundary_coords, normalized_boundary_values], 1)

In [ ]:
bp_top = pf_fields[0].reshape(nx, ny, 3) 
bp_lateral_1 = pf_fields[1].reshape(ny, nz, 3)
bp_lateral_2 = pf_fields[2].reshape(ny, nz, 3) 
bp_lateral_3 = pf_fields[3].reshape(nx, nz, 3) 
bp_lateral_4 = pf_fields[4].reshape(nx, nz, 3) 

In [ ]:
b_bottom = b_bottom / b_norm
bp_top = bp_top / b_norm
bp_lateral_1 = bp_lateral_1 / b_norm
bp_lateral_2 = bp_lateral_2 / b_norm
bp_lateral_3 = bp_lateral_3 / b_norm 
bp_lateral_4 = bp_lateral_4 / b_norm

In [ ]:
b_bottom_path = os.path.join(input_path, "b_bottom.pickle")
bp_top_path = os.path.join(input_path, "bp_top.pickle")
bp_lateral_1_path = os.path.join(input_path, "bp_lateral_1.pickle")
bp_lateral_2_path = os.path.join(input_path, "bp_lateral_2.pickle")
bp_lateral_3_path = os.path.join(input_path, "bp_lateral_3.pickle")
bp_lateral_4_path = os.path.join(input_path, "bp_lateral_4.pickle")
boundary_data_path = os.path.join(input_path, "boundary_data.pickle")

In [ ]:
with open(b_bottom_path,"wb") as f:
    pickle.dump(b_bottom, f)

with open(bp_top_path,"wb") as f:
    pickle.dump(bp_top, f)

with open(bp_lateral_1_path,"wb") as f:
    pickle.dump(bp_lateral_1, f)

with open(bp_lateral_2_path,"wb") as f:
    pickle.dump(bp_lateral_2, f)
    
with open(bp_lateral_3_path,"wb") as f:
    pickle.dump(bp_lateral_3, f)

with open(bp_lateral_4_path,"wb") as f:
    pickle.dump(bp_lateral_4, f)

with open(boundary_data_path,"wb") as f:
    pickle.dump(boundary_data, f)